
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Mechanizmy atencji Neural Turing Machine</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Atencja i rozszerzenia modelu RNN
Przydatne rozszerzenia modelu rekurencyjnego
1. Neural Turing Machine z pamięcią
2. Memory Networks z pamięcią
3. Atencja pozwalająca na skupianie się na fragmentach obrazu (obiektu)
4. Adaptywne obliczenia pozwalające skupić się dłużej na częściach istotnych
5. Neuronowe programowanie

## Mechanizm pamięci
<img src="../nn_figures/memoryNN.png" width="100%"> [distill.pub]
1. typowe RNN tworzą _jeden_ blok stanu gdzie zapamiętują wszystko
2. wyróżnić fragmenty pamięci z której można czytać/pisać
  * co czytać?
  * gdzie pisać?

## Neural Turing Machine
<img src="../nn_figures/NTM.png" width="50%" align="right"> [distill.pub]
NTM składa się z

* kontrolera i pamięci

gdzie każdy alement jest różniczkowalny

---






W jaki sposób ustalić adresowanie pamięci? Gdzie czytać? Gdzie pisać? Zapis/odczyt są rozmyte (blurry).

NTM tworzy __rozkład atencji__ (attention distribution)

* pisze i czyta wszędzie ale w różnym stopniu
* zapis i odczyt na podobieństwo LSTM
* czytanie $$r_t = \sum_i w_t(i)M_t(i),$$ gdzie $w_t(i)$ jest wagą obliczaną przez _głowicę odczytującą_
* pisanie 
$$\begin{align}
\tilde{M}_t(i)&=M_{t-1}(i)\left[1-w_t(i)e_t\right]\\
M_i&=\tilde{M}_t(i)+w_t(i)a_t,
\end{align}$$
gdzie 
  * $e_t$ to sygnał kasujący (erase), $e_{ti}\in(0,1)$
  * $w_t(i)$ jest wagą obliczaną przez _głowicę zapisującą_
  * $e_t$ to sygnał dodawania informacji (add), $a_{ti}\in(0,1)$
  * sygnał jest zerowany jeśli jednocześnie wagi i sygnał kasowania są $1$
  * wszystkie składniki wektorów kasowania i dodawania są niezależne


<img src="../nn_figures/NTM_addressing.jpg" width="35%" align="right">
W jaki sposób ustalić adresowanie?

* oparte na zawartości: model przegląda pamięć w poszukiwaniu miejsc, które pasują do poszukiwania
  * adresowanie przez zawartość jest realizowane (w oryginalnej implementacji) przez odległość kosinusową $$k_t[u,v]=\frac{u\cdot{}v}{\|u\|\,\|v\|}$$
  * $softmax(\beta_tk_t(u,v))$ określa wagę $w_t^c$ (content)
* oparte na położeniu 
  * wyrażenia arytmetyczne $a+b$ przy ewaluacji wymagają _położenia_ wartości dla $a$ i $b$
  $$\begin{align}
  w_t^g&=g_tw_t^c+(1-g_t)w_{t-1}\tag{bramka}\\
  \tilde{w}_t(i)&=\sum_j^{N-1}w_t^g(j)s_t(i-j)\tag{konwolucje}\\
  w_t(i)&=softmax(\tilde{w}_t(i)^{\gamma_t}\tag{wyostrzanie}
  \end{align}$$
  * $w_t^g$ to bramkowanie sygnału; jeśli $g_t=0$, to aktualny sygnał jest ignorowany
  * $s_t$ jest sygnałem pozwalajacym na dopasowanie się do przesunięcia
  * $\gamma_t$ jest sygnałem wyostrzania

Łącznie sygnały $w_t, e_t, a_t$ tworzą rozkład atencji

## Neural Turing Machine
<img src="../nn_figures/NTM_diagram.gif" width="100%"> [distill.pub]

 
NTM jest w stanie działać jak pamięć adresowalna zawartością, powtarzać sekwencje, nawet sortować liczby.

## NTM
1. NTM składa się z pamięci i _kontrolera_
  * parametrami są wielkość pamięci, liczba głowic, etc.
2. Jaka sieć powinna pracować jako kontroler
  * LSTM ma własną pamięć stanu, która może uzupełniać pamięć NTM
    * ukryte aktywacje można porównać do rejestrów
  * możliwa jest sieć typu feedforward


### NTM - zadanie kopiowania
1. NTM dostawał sekwencje wektorów o losowych długościach do 20
2. zadaniem odtwarzanie sekwencji
  * w trakcie odtwarzania NTM _nie dostawał_ żadnych wejść
3. NTM uczy się znacznie szybciej niż model LSTM
  * kontroler feedforward był skuteczniejszy niż kontroler LSTM
  * NTM nie ma problemów z wydłużającymi się sekwencjami, skuteczność LSTM gwałtownie spada powyżej 20
  
  * NTM nauczony dla sekwencji do 20 działął w miarę poprawnie dla dłuższych
    * pojawiał sie problem przesunięcia w sekwencji
    
### NTM także kopiowanie ustaloną liczbę razy, pamięć asocjacyjna, uczenie n-gramów, sortowanie
* NTM ma zwykle sporo mniej parametrów niż odpowiadający mu funkcjonalnie LSTM